Notes while reading through
Python Metaclasses - https://realpython.com/python-metaclasses/

In [12]:
n = 5
d = {'x': 1, 'y': 2}

class Foo:
    pass

x = Foo()

print(type(x))
print(x.__class__)
type(Foo)

<class '__main__.Foo'>
<class '__main__.Foo'>


type

In [13]:
for t in int, float, dict, list, tuple:
    print(type(t))

<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


In [14]:
type(type)

type

In [19]:
# Defining a class using `type` statement
# pattern - type(<name>, <bases>, <dct>)
# name = Name of the class - becomes the __name__ attribute of class
# bases = The base class from which this class should inherit - becomes the __bases__ attribute of the class
# dict = Namespace dictionary containing definitions for the class body - becomes the __dict__ attribute of the class

In [20]:
Foo = type('Foo', (), {})
print(Foo)
print(type(Foo))
print(type(Foo()))

<class '__main__.Foo'>
<class 'type'>
<class '__main__.Foo'>
<class '__main__.Foo'>
<class 'type'>
<class '__main__.Foo'>


In [22]:
# Defining the same class using `class` statement
class Foo:
    pass

x = Foo()
print(Foo)
print(type(Foo))
print(type(x))

<class '__main__.Foo'>
<class 'type'>
<class '__main__.Foo'>


In [23]:
# <bases> is a tuple with single element `Foo` - indicates parent class
# <dict> defines attribute `attr` in the Bar class's namespace dictionary
Bar = type('Bar', (Foo, ), dict(attr=100))
x = Bar()
print(x.attr)
print(x.__class__)
print(x.__class__.__bases__)



100
<class '__main__.Bar'>
(<class '__main__.Foo'>,)


In [24]:
# Defining the same class using `class` attribute
class Bar(Foo):
    attr = 100
    
y = Bar()
print(y.attr)
print(y.__class__)
print(y.__class__.__bases__)

100
<class '__main__.Bar'>
(<class '__main__.Foo'>,)


In [26]:
# This time <bases> is again empty,
# Two objects are placed in the namespace dictionary via the <dct> argument
## First is named `attr`
## Second is named `attr_val` - This becomes a method of the defined class

Foo = type(
    'Foo',
    (),
    {
        'attr': 100,
        'attr_val': lambda x: x.attr
    }

)
x = Foo()
print(x.attr)
print(x.attr_val())

100
100


In [27]:
# Defining the same class using `class` statement
class Foo:
    attr = 100
    def attr_val(self):
        return self.attr

y = Foo()
print(y.attr)
print(y.attr_val())

100
100


In [30]:
# Using a slightly more complex function (defined externally) assigned to `attr_val`
def f(obj):
    print('attr = {}'.format(obj.attr))

Foo = type(
    'Foo',
    (),
    {
        'attr': 100,
        'attr_val': f
    }
)

z = Foo()
print(z.attr)
print(z.attr_val())

100
attr = 100
None


In [33]:
# Same class now defined using `class` statement
def f(obj):
    print('attr = {}'.format(obj.attr))

class Foo:
    attr = 100
    attr_val = f

a = Foo()
print(a.attr)
print(a.attr_val())

100
attr = 100
None


In [34]:
class Foo:
    pass

f = Foo()

# Flow when a new class is created
- Expression `Foo()` creates a new instance of class `Foo`
- As interpreter encounters `Foo()`, following happens:
  - `__call__()` method of `Foo`'s parent class is invoked
  - `__call__()` method in turn invokes following:
    - `__new__()`
    - `__init__()`
- If `Foo` doesn't define `__new__()` and `__init__()`, default methods are inherited from `Foo`'s ancestry
- If `Foo` does define, they override from ancestry - hence we can define custom behaviour when instantiating `Foo`
- Let's define a custom `__new__()` and use it for `Foo`

In [35]:
def new(cls):
    x = object.__new__(cls)
    x.attr = 100
    return x

Foo.__new__ = new
f = Foo()
print(f.attr)

100


In [36]:
g = Foo()
print(g.attr)

100


- You cannot modify `__new__()` on `type` metaclass (if you wanted to customize the behavior **while creating the class**) - Python does not allow this, see example below:

In [37]:
def new(cls):
    x = type.__new__(cls)
    x.attr = 100
    return x

type.__new__ = new

TypeError: can't set attributes of built-in/extension type 'type'

A workaround here is to not mess around `type` metaclass but instead to define your own metaclass with customized `__new__()`, see example:

In [38]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100
        return x

What makes `Meta` a metaclass is that it inherits from `type` which is itself a metaclass

We couldn't modify `type`'s `__new__()` directly, we were able to create a new metaclass `Meta` with a customized `__new__()` method.

The `__new__()` method does following:
- Delegates via `super()` to the `__new__()` method of the parent metaclass (`type`) to actually create a new class
- Assigns the custom attribute `attr` to the class, with value of `100`
- Returns the newly created class

Now let us define a new class `Foo` with metaclass as `Meta`

In [39]:
class Foo(metaclass=Meta):
    pass

Foo.attr
# Foo picks up the `attr` attribute automatically from the metaclass

100

In [40]:
# Let us try some more
class Bar(metaclass=Meta):
    pass
class NewClass(metaclass=Meta):
    pass

Bar.attr, NewClass.attr

(100, 100)

Just as class functions as a template for the creation of objects (instances of that particular class), a metaclass functions as a template for creation of classes.
Metaclasses are sometimes referred to as [class factories](https://en.wikipedia.org/wiki/Factory_(object-oriented_programming/)

# Object Factory

In [41]:
class Foo:
    def __init__(self):
        self.attr = 100

x = Foo()
print(x.attr)

y = Foo()
print(y.attr)

z = Foo()
print(z.attr)

100
100
100


# Class Factory

In [43]:
class Meta(type):
    def __init__(
        cls, name, bases, dct
    ):
        cls.attr = 100

class X(metaclass=Meta):
    pass

print(X.attr)


class Y(metaclass=Meta):
    pass

print(Y.attr)


class Z(metaclass=Meta):
    pass

print(Z.attr)

100
100
100


# Question - Is it really worth it?
## Other ways in which the above can be accomplished


### Simple Inheritance

In [44]:
class Base:
    attr = 100

class X(Base):
    pass

class Y(Base):
    pass

class Z(Base):
    pass

print(X.attr)
print(Y.attr)
print(Z.attr)

100
100
100


### Class Decorator

In [45]:
def decorator(cls):
    class NewClass(cls):
        attr = 100
    return NewClass

@decorator
class X:
    pass

@decorator
class Y:
    pass

@decorator
class Z:
    pass

print(X.attr)
print(Y.attr)
print(Z.attr)

100
100
100


# Conclusion

As Tim Peters suggests, metaclasses can easily veer into the realm of being a “solution in search of a problem.” It isn’t typically necessary to create custom metaclasses. If the problem at hand can be solved in a simpler way, it probably should be.
Still, it is beneficial to understand metaclasses so that you understand Python classes in general and can recognize when a metaclass really is the appropriate tool to use.